# Weekly Project 5
## Global Registration implementation.
## Task 1
Today your project is to implement a global registration algorithm.

It should be able to roughly align two pointclouds.
1. Implement global registration
2. Can you fit **r1.pcd** and **r2.pcd**?
3. Can you fit **car1.ply** and **car2.ply**?
These are in the *global_registration* folder



## Task 2 (Challange)
Challanges attempt either or both:
- Implement local registration.

- Attempt to reconstruct the car from the images in *car_challange* folder.

You can use the exercises from monday as a starting point.

In [14]:
###
# Add code here
import open3d as o3d
import numpy as np
import copy
from sklearn.neighbors import KDTree

# helper function for drawing if you want it to be more clear which is which set recolor=True
def draw_registrations(source, target, transformation = None, recolor = False):
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    if(recolor):
        source_temp.paint_uniform_color([1, 0.706, 0])
        target_temp.paint_uniform_color([0, 0.651, 0.929])
    if(transformation is not None):
        source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target_temp])

In [2]:
source = o3d.io.read_point_cloud("global_registration/r1.pcd")
#target = o3d.io.read_point_cloud("ICP/r2.pcd")
target = o3d.io.read_point_cloud("global_registration/r2.pcd")

# Show models side by side
draw_registrations(source, target)

In [3]:
def preprocess_point_cloud(pcd, voxel_size):
    print(":: Downsample with a voxel size %.3f." % voxel_size)
    pcd_down = pcd.voxel_down_sample(voxel_size)

    radius_normal = voxel_size * 2
    print(":: Estimate normal with search radius %.3f." % radius_normal)
    pcd_down.estimate_normals(
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normal, max_nn=30))

    radius_feature = voxel_size * 5
    print(":: Compute FPFH feature with search radius %.3f." % radius_feature)
    pcd_fpfh = o3d.registration.compute_fpfh_feature(
        pcd_down,
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius_feature, max_nn=100))
    return pcd_down, pcd_fpfh

In [4]:
voxel_size=0.05 #越大越稀疏

source_down,source_fpfh = preprocess_point_cloud(source,voxel_size)
target_down, target_fpfh = preprocess_point_cloud(target,voxel_size)
draw_registrations(source_down, target_down,recolor = True)

:: Downsample with a voxel size 0.050.
:: Estimate normal with search radius 0.100.
:: Compute FPFH feature with search radius 0.250.
:: Downsample with a voxel size 0.050.
:: Estimate normal with search radius 0.100.
:: Compute FPFH feature with search radius 0.250.


In [21]:
source_fpfh.data

array([[ 0.        ,  0.        ,  0.        , ...,  0.09281381,
        16.32878834, 24.50135888],
       [ 0.        ,  0.        ,  0.        , ...,  0.10635172,
         4.08235868, 11.78067895],
       [ 5.0265493 ,  5.75492718,  5.50880446, ...,  1.51336887,
         8.77004199,  4.8595878 ],
       ...,
       [47.38065107, 49.40532664, 54.58096243, ...,  2.04928996,
        13.35553641, 11.39145711],
       [30.54388681, 32.7701926 , 31.95481487, ...,  1.60809268,
        12.99884273,  8.58088671],
       [ 3.97823636,  3.65499865,  4.0724794 , ...,  0.48399374,
        12.1665199 ,  0.72662031]])

In [12]:
source_fpfh.dimension

<bound method PyCapsule.dimension of registration::Feature class with dimension = 33 and num = 4760
Access its data via data member.>

In [16]:
target_fpfh.dimension

<bound method PyCapsule.dimension of registration::Feature class with dimension = 33 and num = 3440
Access its data via data member.>

In [22]:
source_down.points

std::vector<Eigen::Vector3d> with 4760 elements.
Use numpy.asarray() to access data.

In [10]:
np.asarray(source_down.points)

array([[2.23442745, 1.33203125, 2.36328125],
       [2.21174844, 1.35197699, 2.37628685],
       [2.24693085, 1.29708717, 2.37313138],
       ...,
       [0.859375  , 1.44921875, 2.41223095],
       [1.15630122, 2.89035268, 2.38624587],
       [1.15736005, 1.29958513, 2.39776017]])

In [ ]:
	
def execute_global_registration(source_down, target_down, source_fpfh,
                                target_fpfh, voxel_size):
    distance_threshold = voxel_size * 1.5
    print(":: RANSAC registration on downsampled point clouds.")
    print("   Since the downsampling voxel size is %.3f," % voxel_size)
    print("   we use a liberal distance threshold %.3f." % distance_threshold)

    point_to_point =  o3d.registration.TransformationEstimationPointToPoint(False)
    point_to_plane =  o3d.registration.TransformationEstimationPointToPlane()

    corr_length = 0.9
    distance_threshold = voxel_size * 1.5

    c0 = o3d.registration.CorrespondenceCheckerBasedOnEdgeLength(corr_length)
    c1 = o3d.registration.CorrespondenceCheckerBasedOnDistance(distance_threshold)
    c2 = o3d.registration.CorrespondenceCheckerBasedOnNormal(0.095)

    checker_list = [c0,c1,c2]

    result = o3d.registration.registration_ransac_based_on_feature_matching(
        source_down, target_down, 
        source_fpfh, target_fpfh,
        distance_threshold,
        point_to_point,
        #point_to_plane,
        checkers = checker_list)
    return result

In [46]:
import random 

In [60]:
    sf=source_fpfh.data.T
    tf=target_fpfh.data.T
    tree = KDTree(sf)
    nearest_dist, nearest_ind = tree.query(tf) 

In [61]:
tf.shape

(3440, 33)

In [62]:
nearest_ind

array([[1904],
       [2117],
       [ 315],
       ...,
       [ 126],
       [3728],
       [4333]])

In [63]:
len(tf)

3440

In [70]:
random_list=[]
for i in range(3):
    x=random.randint(0,len(tf))
    random_list.append(x)

In [71]:
    pairs=[random_list;nearest_ind[random_list]]
    tp=np.asarray(tf[random_list])
    sp=np.asarray(sf[)  
    s_centroid= np.mean(sp,axis=0)
    t_centroid = np.mean(tp,axis=0)

    sp_centralized = sp-s_centroid
    tp_centralized = tp-t_centroid

    cov_matrix = np.sum(sp_centralized*tp_centralized.T)

SyntaxError: invalid syntax (<ipython-input-71-05e205f10493>, line 1)

In [67]:
tp

array([[2.01788751e-01, 1.92633690e-01, 2.30187240e+00, 1.13100283e+01,
        2.61211225e+00, 1.69541382e+02, 5.67510063e+00, 1.46636709e+00,
        1.88463353e+00, 4.62065085e+00, 1.93430827e-01, 1.29631402e+00,
        1.75643134e+00, 4.39518678e+00, 2.04181335e+00, 1.97406299e+01,
        1.41658707e+02, 1.97973891e+01, 2.44100454e+00, 2.43295482e+00,
        1.57319193e+00, 2.86637698e+00, 6.01290531e-01, 2.11208785e+00,
        1.28475912e+00, 2.89630036e+00, 5.66438445e+01, 1.05576512e+02,
        9.06381855e+00, 3.70623978e+00, 2.77464179e+00, 4.58113762e+00,
        1.07593684e+01],
       [1.53430804e-01, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        2.99395410e-01, 1.90443471e+02, 9.10370257e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00, 8.18068459e-02, 9.06597115e+00,
        1.82376160e+02, 8.14888663e+00, 3.27175821e-01, 0.00000000e+00,
        0.00000000e+00, 0.00000000e+00,

In [ ]:
    sp=np.asarray(source_down.points)
    tp=np.asarray(target_down.points)
    s_centroid= np.mean(sp,axis=0)
    t_centroid = np.mean(tp,axis=0)

    sp_centralized = sp-s_centroid
    tp_centralized = tp-t_centroid

    cov_matrix = np.sum(sp_centralized*tp_centralized.T)

In [ ]:
u, s, vh = np.linalg.svd(a, full_matrices=True)

In [34]:
sp

array([[2.23442745, 1.33203125, 2.36328125],
       [2.21174844, 1.35197699, 2.37628685],
       [2.24693085, 1.29708717, 2.37313138],
       ...,
       [0.859375  , 1.44921875, 2.41223095],
       [1.15630122, 2.89035268, 2.38624587],
       [1.15736005, 1.29958513, 2.39776017]])

In [35]:
s_centroid

array([1.5479294 , 2.13108925, 1.91847977])

In [25]:
def  my_ransac( 
        source_down, target_down, 
        source_fpfh, target_fpfh,
        distance_threshold):

    # pair the correspondant features
    sf=source_fpfh.data.T
    tf=target_fpfh.data.T
    tree = KDTree(sf)
    nearest_dist, nearest_ind = tree.query(tf, k=2)   

    # in loop:
    #   choose random three points
    #   calculate the transform matrix
    #   apply the transform matrix on all points
    #   calculate the MSE of whole points
    sp=np.asarray(source_down.points)
    tp=np.asarray(target_down.points)
    s_centroid= mean(sp)

    random.randint(0,len(sp))
    
    # choose the best one as transformation

    source_fpfh.data 
    err=0
    err_min=1000000
    for i in range(source.point):
         err += (source.points_[c[0]] - target.points_[c[1]]).squaredNorm();
         if err<err_min:

    return transMatrix         

IndentationError: expected an indented block (<ipython-input-25-9fc7e3311a4a>, line 32)